# Projeto 2 - Ciência dos Dados

### Proposta geral do Projeto 2: Prevendo _targets_ baseando-se em um grupo de _features_. 
### Turma A

**Membros:**
-
+ Alessandra Yumi Carvalho Ogawa
+ André Brito
+ Arthur Cisotto Machado
+ Camila Bernardi Moniz


### Importando as Bibliotecas Necessárias

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats 
import itertools
import plotly.express as px
import seaborn as sns
import numpy as np
import os
import re
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


## A. Introdução  
    "Detalhar objetivo escolhido para trabalhar neste projeto juntamente com
    descrição da base de dados. Pesquise trabalhos na literatura que discutam o
    tema escolhido. Para trabalhos acadêmicos, acesse
    https://scholar.google.com.br/. Guarde as referências estudadas para citá-las
    no seu projeto."

## B. Minerando Dados e Características do Dataset  
    Se necessário, faça filtro na base de dados tanto de linhas como de colunas
    em prol do objetivo traçado anteriormente. Descreva as variáveis finais que serão utilizadas a partir deste ponto.
    Faça análise descritiva detalhada das variáveis, norteado pelo objetivo do
    problema. Aqui, é interessante entender como sua variável target se
    comporta cruzada com cada feature. Note que ao cruzar duas variáveis, pode obter o cruzamento entre: duas variáveis quantitativas; duas variáveis
    qualitativas; ou uma de cada tipo. Cada cruzamento irá exigir ferramentas
    descritivas distintas. A tabela a seguir apresenta algumas ferramentas descritivas vistas no curso:

In [ ]:
d = {'': ['Duas variáveis qualitativas', 'Duas variáveis quantitativas', 'Uma variável de cada'], 'Ferramentas estatísticas': ['Tabela cruzadas (com uso de normalize adequado ao problema); Gráficos de barras (empilhados ou stacked); entre outras', 'Medidas de associação; Gráfico de dispersão; entre outras', 'Medidas-resumo da variável quantitativa segmentando por rótulo da variável qualitativa; Histograma (ou boxplot) da variável quantitativa segmentando por rótulo da variável qualitativa; entre outras']}
df = pd.DataFrame(data=d)
pd.set_option('display.max_colwidth', None)
df.style.hide_index()

    Storytelling com dados: encontre uma representação gráfica que descreva
    bem os seus dados e que também favoreça no storytelling que pretende
    fazer ao explicar sua linha de raciocínio às outras pessoas (seja em formato
    escrito ou em apresentação). 

### Importando o Dataset

In [ ]:
data = pd.read_csv('user_fake_authentic_4class.csv')
data.head(10)

### Legenda do Dataset:  

In [ ]:
d1 = {'Nome da Variável': ['pos','flw', 'flg', 'bl', 'pic', 'lin', 'cl','cz','ni','erl', 'erc', 'lt', 'hc', 'pr', 'fo', 'cs', 'pi', 'class'], 'Nome da Feature': ['Número de posts', 'Número de seguidores', 'Número contas seguindo', 'Tamanho da Biografia', 'Disponibilidade de Foto', 'Disponibilidade de Link', 'Tamanho médio da Legenda', 'Legenda Zero', 'Porcentagem Sem Imagem', 'Taxa de Engajamento (Like)', 'Taxa de Engajamento (Comentário)', 'Porcentagem de Localização', 'Média de uso de Hashtags', 'Palavras-Chave Promocionais', 'Palavras-Chave Seguidores', 'Similaridade por Cosseno', 'Intervalo de Posts', 'Classe de Usuários'], 'Descrição': ['Número total de posts que o usuário já fez', 'Número de seguidores', 'Número de contas que o usuário segue', 'Número de caracteres da biografia do usuário', 'Valor 0 se o usuário não tem foto de perfil, 1 caso contrário', 'Valor 0 se o usuário não tem Link externo no perfil, 1 caso contrário', 'Número médio de caracteres nas legendas dos posts do usuário', 'Porcentagem (0.0 a 1.0) de legendas que possuem quase 0 (<=3) caracteres', 'Porcentagem (0.0 a 1.0) de posts que não são do tipo imagem (Existem 3 tipos de mídia em um post no Instagram, imagem, vídeo e carrosel)', 'Número de Likes dividido pelo Número de Seguidores', 'Número de Comentários dividiodo pelo Número de Seguidores', 'Porcentagem (0.0 a 1.0) de posts com tag de localização', 'Número médio de hashtags usadas em um post', 'Número médio de palavras-chave promocionais em hashtags (por exemplo, regrann, contest, repost, giveaway, mention, share, give away, quiz, etc)', 'Número médio de palavras-chave procurando seguidores em hashtags (por exemplo, follow, like, folback, follback, f4f, etc)', 'Medida da similaridade de entre dois vetores (posts) num espaço vetorial que avalia o valor do cosseno do ângulo compreendido entre eles.', 'Intervalo de tempo médio entre posts (em horas)', 'Classes 2-class: r (usuários reais/autenticados), f (Usuário fake / Comprou seguidores) | Classes 4-class: r (usuários reais/autenticados), a (Usuário fake Ativo), i (Usuário fake Inativo), s (Usuário fake Spam). Note que as 3 classes de usuário fake (a, i, s) foram determinadas por avaliadores humanos.' ]}
df1 = pd.DataFrame(data=d1)
pd.set_option('display.max_colwidth', None)
df1.style.hide_index()


## Análise Exploratória

### Relação entre features
Primeiramente, verificamos se existem correlações fortes entre features diferentes.  
Se existirem 2 features diferentes fortemente relacionadas podemos descartar alguma delas para facilitar a análise dos dados.

In [ ]:
data.corr()


In [ ]:
x=data.corr()
lista = x.values.tolist()
fig = px.imshow(lista,  labels=dict(x="Features", y="Features", color="Correlação"),
                x=['pos', 'flw', 'flg', 'bl', 'pic', 'lin', 'cl', 'cz', 'ni', 'erl', 'erc', 'lt', 'hc', 'pr', 'fo', 'cs', 'pi'],
                y=['pos', 'flw', 'flg', 'bl', 'pic', 'lin', 'cl', 'cz', 'ni', 'erl', 'erc', 'lt', 'hc', 'pr', 'fo', 'cs', 'pi'],
                zmax = 1, zmin = -1)
fig.show()

#### Achando as maiores e menores correlações:

In [ ]:
lista_corrs = list(itertools.chain(*lista))

lista_corrs_sem_1 = list(filter(lambda x: x != 1 and x != -1, lista_corrs))

maximo = max(lista_corrs_sem_1)
minimo = min(lista_corrs_sem_1)

print(f"A maior correlação é {maximo}")
print(f"A menor correlação é {minimo}")

#### Como pode-se observar na célula de cima as maiores e menores correlações não são suficiente significativas a ponto de se destacarem de certo modo. Por isso, conclui-se que não será possível o descarte de nenhuma _feature_.

#### Considerando-se que uma correlação só poderia ser considerada forte quando acima de 0.8.

### Relação entre a variável target (Class) e as features

In [ ]:
perfis_reais = data.loc[data['class']=='r']
perfis_fake_ativo = data.loc[data['class']=='a']
perfis_fake_inativo = data.loc[data['class']=='i']
perfis_fake_spam = data.loc[data['class']=='s']

**Observação:** em muitos dos histogramas à seguir foi utilizado algum tipo de limite para os eixos Y e X, procurando ter melhor visualização da distribuição de densidade. O estabelecimento de limites foram nescessários em muitos casos vistos que as classificações de usuários se diferem muito entre si, o que contribuiu para a aparição de alguns _outliers_ dificultando a interpretação visual dos gráficos.

+ #### Relação entre Class e Número de posts	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="pos",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['pos'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['pos'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['pos'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['pos'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlim(0,10000)
plt.ylim(0,0.001)
plt.xlabel('número de posts')
plt.ylabel('densidade')
plt.show()

Em relação a _feature_ posts (POS) muito pode-se concluir do histograma. Fica claro por contraste visual que usuários que possuem uma extrema quantidade de posts, com grande probabilidade serão classificados como _spam_; o que já era de se esperar do comportamento desse tipo de usuário de Instagram. Outro ponto de muito destaque, é a concentração de usuários inativos na área que representa uma quantidade baixíssima de poasts; comportamento também esperado por esse tipo de usuário.

+ #### Relação entre Class e Número de seguidores	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="flw",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['flw'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['flw'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['flw'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['flw'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlim(0,30000)
plt.ylim(0,0.0002)
plt.xlabel('número de seguidores')
plt.ylabel('densidade')
plt.show()

Em relação a _feature_ número de seguidores (FLW) verifica-se que tanto os usuários spam quanto os reais apresentam um comportamento similar, o que pode apresentar uma complicação na hora de classificar futuramente demais usuários. Apesar disso, verifica-se comportamentos distintos para usuários inativos e ativos fake; comportamentos esperados dada as características de atividade desses usuários.

+ #### Relação entre Class e Número contas seguindo

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="flg",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['flg'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['flg'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['flg'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['flg'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlim(0,10000)
plt.ylim(0,0.001)
plt.xlabel('número de contas que segue')
plt.ylabel('densidade')
plt.show()

Analisando o primeiro gráfico a respeito da _feature_ número de contas seguindo (FLG) percebe-se que as medianas (representadas pela linha horizontal verde) são diferentes para cada uma das classificações. Isso leva a concluir que a faixa de número de seguidores em que uma conta de Instagram se classificar será importante para a classificação final de tipo de usuário.

+ #### Relação Class e Tamanho da Biografia	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="bl",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['bl'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['bl'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['bl'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['bl'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlabel('número de caracteres da bio')
plt.ylabel('densidade')
plt.show()

A _feature_ tamanho da biografia (BL) não parece ser um elemento decisivo na classificação, dada a proximidade de suas medianas observadas no primeiro gráfico. Apesar disso, observando o histograma fica claro que há certa diferença de concentração entre as classificações, porém nada muito decisivo por se tratarem de _outliers_.

+ #### Relação Class e Disponibilidade de Foto	

In [ ]:
perfis_sem_foto = data.loc[data['pic']== 0]
perfis_com_foto = data.loc[data['pic']== 1]

perfis_reais_com_foto = perfis_com_foto.loc[perfis_com_foto['class']=='r']
perfis_fake_ativo_com_foto = perfis_com_foto.loc[perfis_com_foto['class']=='a']
perfis_fake_inativo_com_foto = perfis_com_foto.loc[perfis_com_foto['class']=='i']
perfis_fake_spam_com_foto = perfis_com_foto.loc[perfis_com_foto['class']=='s']

perfis_reais_sem_foto = perfis_sem_foto.loc[perfis_sem_foto['class']=='r']
perfis_fake_ativo_sem_foto = perfis_sem_foto.loc[perfis_sem_foto['class']=='a']
perfis_fake_inativo_sem_foto = perfis_sem_foto.loc[perfis_sem_foto['class']=='i']
perfis_fake_spam_sem_foto = perfis_sem_foto.loc[perfis_sem_foto['class']=='s']

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
perfis = ['r', 'a', 'i', 's']
quantidade = [perfis_reais_com_foto.shape[0]/perfis_com_foto.shape[0], perfis_fake_ativo_com_foto.shape[0]/perfis_com_foto.shape[0], perfis_fake_inativo_com_foto.shape[0]/perfis_com_foto.shape[0], perfis_fake_spam_com_foto.shape[0]/perfis_com_foto.shape[0]]
ax.bar(perfis,quantidade, color=['crimson', 'midnightblue', 'lime', 'yellow'])
plt.title('Com foto de perfil')
plt.ylabel('densidade')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
perfis = ['r', 'a', 'i', 's']
quantidade = [perfis_reais_sem_foto.shape[0]/perfis_sem_foto.shape[0], perfis_fake_ativo_sem_foto.shape[0]/perfis_sem_foto.shape[0], perfis_fake_inativo_sem_foto.shape[0]/perfis_sem_foto.shape[0], perfis_fake_spam_sem_foto.shape[0]/perfis_sem_foto.shape[0]]
ax.bar(perfis,quantidade,color=['crimson', 'midnightblue', 'lime', 'yellow'])
plt.title('Sem foto de perfil')
plt.ylabel('densidade')
plt.show()

A partir dos graficos, conclui-se que perfis sem disponibilidade de fotos tendem a serem classificados como fakes inativos,comportamento esperado já que fakes inativos." é uma classificação refetente a usuários que estão com a conta parada.Enquanto perfis com disponibilidade de fotos variam entre perfis de usuários ativos, fakes ativos e spam.

+ #### Relação Class e Disponibilidade de Link	

In [ ]:
perfis_sem_link = data.loc[data['pic']== 0]
perfis_com_link= data.loc[data['pic']== 1]

perfis_reais_com_link = perfis_com_link.loc[perfis_com_link['class']=='r']
perfis_fake_ativo_com_link = perfis_com_link.loc[perfis_com_link['class']=='a']
perfis_fake_inativo_com_link = perfis_com_link.loc[perfis_com_link['class']=='i']
perfis_fake_spam_com_link = perfis_com_link.loc[perfis_com_link['class']=='s']

perfis_reais_sem_link = perfis_sem_link.loc[perfis_sem_link['class']=='r']
perfis_fake_ativo_sem_link = perfis_sem_link.loc[perfis_sem_link['class']=='a']
perfis_fake_inativo_sem_link = perfis_sem_link.loc[perfis_sem_link['class']=='i']
perfis_fake_spam_sem_link = perfis_sem_link.loc[perfis_sem_link['class']=='s']

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
perfis = ['r', 'a', 'i', 's']
quantidade = [perfis_reais_com_link.shape[0]/perfis_com_link.shape[0], perfis_fake_ativo_com_link.shape[0]/perfis_com_link.shape[0], perfis_fake_inativo_com_link.shape[0]/perfis_com_link.shape[0], perfis_fake_spam_com_link.shape[0]/perfis_com_link.shape[0]]
ax.bar(perfis,quantidade, color=['crimson', 'midnightblue', 'lime', 'yellow'])
plt.title('Com link no perfil')
plt.ylabel('densidade')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
perfis = ['r', 'a', 'i', 's']
quantidade = [perfis_reais_sem_link.shape[0]/perfis_sem_link.shape[0], perfis_fake_ativo_sem_link.shape[0]/perfis_sem_link.shape[0], perfis_fake_inativo_sem_link.shape[0]/perfis_sem_link.shape[0], perfis_fake_spam_sem_link.shape[0]/perfis_sem_link.shape[0]]
ax.bar(perfis,quantidade, color=['crimson', 'midnightblue', 'lime', 'yellow'])
plt.title('Sem link perfil')
plt.ylabel('densidade')
plt.show()

A partir dos gráficos, nota-se que perfis que não possuem links tendem a estarem associados a perfis de fakes inativos, comportamento esperado pela característica da classificação de serem contas não mais acessadas. Enquanto perfis que possuem links tendem a estarem associados a usuários fakes ativos, spam e usuários reais (majoritariamente).


+ #### Relação Class e Tamanho médio da Legenda	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="cl",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['cl'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['cl'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['cl'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['cl'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,2500)
plt.xlabel('número médio de caracteres da legenda')
plt.ylabel('densidade')

plt.show()

A partir do gráfico, conclui-se que todas as classificações possuem um número médio de caracteres próximo, porém a classificação spam tende a ter um número de caracteres mais alto, comportamento que se justifica na caractéristica de usuários "spam" ligada a propagação de informação em massa.

+ #### Relação Class e Legenda Zero	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="cz",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['cz'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['cz'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['cz'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['cz'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlabel('Porcentagem (0.0 a 1.0) de legendas que possuem quase 0 (<=3) caracteres')
plt.ylabel('densidade')
plt.show()

A partir do gráfico, conclui-se que usuários ativos tendem a ter menos legenda zero, devido a atividade do usuário na rede  característica de sua classificação, entretanto, podemos observar sua presença nos extremos próximos de 1 da legenda zero, correpondente a "outliers". Ainda, observa-se que a mediana de usuários reais indica a alta presença de legendas (exceto por pontos outliers concentrados próximos a 1). Por fim, usuários spam possuem a maior mediana, ou seja, tendem mais a não ter legenda e usuários inativos encontram-se distribuídos ao longo do parâmetro, ou seja, não é uma feature que ajude a determinar este tipo de classificação.

+ #### Relação Class e Porcentagem Sem Imagem

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="ni",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['ni'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['ni'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['ni'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['ni'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlabel('Porcentagem (0.0 a 1.0) de posts que não são do tipo imagem')
plt.ylabel('densidade')
plt.show()

Analisando o primeiro gráfico e o histograma a respeito da _feature_ de porcentagem de posts que não são tipo imagem (NI) percebe-se que essa variável tem comportamento semelhante para todos os tipos de perfil, no entanto perfis fakes inativos tem uma característica discrepante que é uma mediana igual a 0.

+ #### Relação Class e Taxa de Engajamento (Like)	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="erl",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['erl'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['erl'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['erl'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['erl'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])
plt.xlim(0,4000)
plt.xlabel('Número de Likes dividido pelo Número de Seguidores')
plt.ylabel('densidade')
plt.show()

A _feature_ Taxa de Engajamento em Likes (ERL) não parece ser um elemento decisivo na classificação, dada a proximidade de suas medianas observadas no primeiro gráfico. Apesar disso, observando o histograma fica claro que há certa diferença de concentração entre as classificações, porém nada muito decisivo por se tratarem de _outliers_.

+ #### Relação Class e Taxa de Engajamento (Comentário)	

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="erc",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['erc'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['erc'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['erc'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['erc'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,100)
plt.xlabel('Número de Comentários dividido pelo Número de Seguidores')
plt.ylabel('densidade')
plt.show()

A _feature_ Taxa de Engajamento em Comentários (ERC) não parece ser um elemento decisivo na classificação, dada a proximidade de suas medianas observadas no primeiro gráfico. Apesar disso, observando o histograma fica claro que há certa diferença de concentração entre as classificações, porém nada muito decisivo por se tratarem de _outliers_.

+ #### Relação Class e Porcentagem de Localização

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="lt",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['lt'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['lt'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['lt'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['lt'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,1)
plt.xlabel('Porcentagem (0.0 a 1.0) de posts com tag de localização')
plt.ylabel('densidade')
plt.show()

A _feature_ Porcentagem de Localização (LT) parece ser um elemento decisivo na classificação, dado que a mediana observada no primeiro gráfico dos perfis reais é significativamente maior do que os perfis fakes. Observa-se também que as medianas dos perfis fakes inativos e spams são iguais a 0%.

+ #### Relação Class e Média de uso de Hashtags	

In [ ]:
data.boxplot(column="hc",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['hc'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['hc'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['hc'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['hc'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,15)
plt.xlabel('Número médio de hashtags usadas em um post')
plt.ylabel('densidade')
plt.show()

Sobre a média de uso de hashtags (HC) o primeiro gráfico indicam que há uma grande quantidade de _outiliers_ em todas as classificações. Talvez porque o uso ou não desse símbolo parta de uma decisão mais pessoal do usuário. Apesar disso vale destacar que pelo histograma observa-se um maior uso de hashtags pelas contas spam.

+ #### Relação Class e Palavras-Chave Promocionais

In [ ]:
data.boxplot(column="hc",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['pr'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['pr'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['pr'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['pr'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,2.5)
plt.xlabel('Número médio de palavras-chave promocionais em hashtags')
plt.ylabel('densidade')
plt.show()

Sobre a _feature_ palavras-chave promocionais (HC) percebe-se pelo histograma que a grande maioria ocorre em contas spam; comportamento esperado por esse tipo de usuário. Além disso, vale distacar a quantidade de _outiliers_ nessa _feature_ verificada visualmente pelo primeiro gráfico; a coluna de pontos indica uma grande dispersão variada, o que talvez indique que essa variável não seja tão decisiva no momento da classificação.

+ #### Relação Class e Palavras-Chave Seguidores

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="fo",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['fo'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['fo'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['fo'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['fo'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,6)
plt.ylim(0,5)
plt.xlabel('Número médio de palavras-chave procurando seguidores em hashtags')
plt.ylabel('densidade')
plt.show()

Em relação à _feature_ palavras-chave seguidores (FO) percebe-se pelo histograma maior ocorrência em perfis ativos, abrangindo-se também para spams. Esse elemento também pode ser considerado uma importante _featrure_ para classificação visto que a rpocura por seguidores em rede sociais pode dizer muito sobre o estado de um usuário.

+ #### Relação Class e Similaridade por Cosseno

In [ ]:
data.boxplot(column="cs",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['cs'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['cs'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['cs'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['cs'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,1)
plt.ylim(0,5)
plt.xlabel('Similaridade por Cosseno')
plt.ylabel('densidade')
plt.show()

+ #### Relação Class e Intervalo de Posts

In [ ]:
plt.figure(figsize=(25,25))
data.boxplot(column="pi",by="class")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))
perfis_reais['pi'].plot.hist(color=('crimson'), alpha=0.6, density=True)
perfis_fake_ativo['pi'].plot.hist(color=('midnightblue'), alpha=0.6, density=True)
perfis_fake_inativo['pi'].plot.hist(color=('lime'), alpha=0.6, density=True)
perfis_fake_spam['pi'].plot.hist(color=('yellow'), alpha=0.6, density=True)
plt.legend(['r', 'a', 'i', 's'])

plt.xlim(0,10000)
plt.xlabel('Intervalo de tempo médio entre posts (em horas)')
plt.ylabel('densidade')
plt.show()

Observando o histograma fica muito claro o comportamento dos usuários spam em relação ao intervalo entre posts (PI) quando comparado com os outros. Em paralelo, o usuário spam possui um intervalo de tempo entre publicações significament menores; um comportamento esperado desse tipo de usuário. Outro comportamento comprovado pelo histograma que também já era esperado é o longo intervalo entre posots por parte dos usuários inativos.

## C. Modelos de Predição 
    Descreva e justifique sua escolha de pelo menos DOIS modelos de predição. Exemplos de uso de modelos neste trabalho:
    https://paulovasconcellos.com.br/10-skills-que-voc%C3%AA-precisar%C3%A1-para-trabalhar-com-data-science-em-2020-d1fe0da75914
    Mas você pode usar outros que
    fizerem mais sentido para o seu problema. Nesta etapa, ajuste cada modelo
    preditivo apenas a uma parte da base de dados chamada de treinamento. A
    validação do modelo está descrita no próximo subitem.

In [ ]:
dataToTree = pd.read_csv('user_fake_authentic_4class.csv')

dataToTree['class'] = dataToTree['class'].replace(['r', 'a', 'i', 's'], [0, 1, 2, 3])

dataToTree['class'].value_counts()

#? Label encoder

In [ ]:
x = dataToTree.drop("class", axis=1)
y = dataToTree["class"]

Xtrain, Xval, Ytrain, Yval = train_test_split(x, y, test_size=0.5, random_state=0)

trees = RandomForestClassifier(n_estimators=1000,random_state=0, n_jobs=-1)
trees.fit(Xtrain, Ytrain)

p = trees.score(Xval, Yval)

#! Célula 94
#! Cálculo da ACURÁCIA
p*100 # 90% de acerto do modelo

# Outras métricas:
#? Precision 
#? Matriz de confusão
#?

#! Usar a mesma estratégia em ambos os modelos

In [ ]:
decision_tree = tree.DecisionTreeClassifier(max_depth=3)
decision_tree = decision_tree.fit(Xtrain, Ytrain)
pred = decision_tree.predict(Xval)
print('Accuracy of ', accuracy_score(pred, Yval))*100


In [ ]:
print(cross_val_score(decision_tree, Xval, Yval, cv = 10)) 

In [ ]:
# funcao para classificar os dados com input do usuario 
lista = [#todas as variaves na mesma ordem do treinamento]
trees.predict(lista)

## D. Processo e Estatísticas de Validação
    
    Para os modelos preditivos que foram desenvolvidos no item anterior, é
    necessário calcular medidas que informam a performance de cada modelo
    ajustado. Assim, para cada modelo preditivo, faça:
     
    o  Divida a base de dados na parte treinamento e na parte teste. Use a
    parte treinamento para estimar cada modelo preditivo. 
    o  Estude as medidas que permitem validar que seu modelo de previsão
    está funcionando bem. Veja alguns exemplos nos links a seguir: link 1,
    link 2 e link 3 (este apenas se target for quantitativo). Escolha medidas
    de performance para os modelos de predição feitos em seu projeto e
    compare-as após calcular tanto predizer a variável usando os dados de
    treinamento como para a parte dos dados teste (o mais importante). 
    o  Discuta se essas duas medidas se comportam de forma semelhante
    para as duas partes de dados. Leia o texto disponível aqui para
    compreender overfitting e underfitting e refinar senso crítico para
    discutir sobre as medidas calculadas. 
    o  Extra: Faça o processo de Validação Cruzada utilizando
    também 10 ciclos e calcule a performance média e desvio
    padrão das duas medidas R2 e RMS tanto para a parte
    treinamento como para a parte teste. Discuta com riqueza de
    detalhes.

## E. Conclusão  
    Faça conclusão final com detalhes levando em consideração todas as
    interpretações realizadas no decorrer do projeto.

## F. Referências Bibliográficas    
    Todas as pesquisas feitas e estudadas que foram relevantes para o
    desenvolvimento devem ser citadas no projeto. 